In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ner-data/ner_dataset.csv


In [2]:
!pip install pandas numpy scikit-learn keras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [3]:
# !pip install pandas numpy scikit-learn keras tensorflow
# Used GPU P100 (Kaggle Kernel)

import logging
import os
import random
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, f1_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import traceback
import spacy
from spacy import displacy

# Configure logging
log_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'logs')
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(log_dir, 'app.log'),
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s]: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
'''
# Load the dataset
data_directory = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'data')
dataset_path = os.path.join(data_directory, 'ner_dataset.csv')
if os.path.exists(dataset_path):
    df_ner = pd.read_csv(dataset_path, encoding="latin1")
else:
    logging.error(f"Dataset not found at: {dataset_path}")
'''

# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available!!")
    device = '/gpu:0'
else:
    print("GPU not available, using CPU instead!")
    device = '/cpu:0'


df_ner = pd.read_csv('/kaggle/input/ner-data/ner_dataset.csv', encoding="latin1")

# Drop unnecessary columns
df_ner = df_ner.drop(columns=["POS"])

print(df_ner)

# we'll preprocess the data by creating a function to transform the dataset into sentences and corresponding labels:

def preprocess_data(data):
    """
    Preprocesses the NER dataset into sentences and corresponding labels.

    Args:
    data: pandas DataFrame containing the NER dataset

    Returns:
    sentences: list of lists containing words for each sentence
    labels: list of lists containing NER labels for each sentence
    """
    sentences = []
    labels = []
    current_sentence = []
    current_labels = []

    for index, row in data.iterrows():
        # If it's the start of a new sentence
        if pd.isnull(row['Sentence #']):
            sentences.append(current_sentence)
            labels.append(current_labels)
            current_sentence = []
            current_labels = []
        else:
            current_sentence.append(row['Word'])
            current_labels.append(row['Tag'])
    
    print("Preprocessing is Completed !!")
    return sentences, labels

sentences, labels = preprocess_data(df_ner)

# Now, let's split the dataset into train, validation, and test sets:

# Split the data into train and test sets (80% train, 20% test)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)

# Further split the train set into train and validation sets (80% train, 20% validation)
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_sentences, train_labels, test_size=0.2, random_state=42)

# With the data preprocessed and split, we can now move on to building the baseline model. We'll use a simple LSTM-based model for this:

# Tokenize words and labels
words = list(set(df_ner["Word"].values))
n_words = len(words)

tags = list(set(df_ner["Tag"].values))
n_tags = len(tags)

# Create mappings for words and tags
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

# Padding sequences
max_len = 50
X_train = [[word2idx[w] for w in s] for s in train_sentences]
X_train = pad_sequences(maxlen=max_len, sequences=X_train, padding="post", value=n_words-1)

y_train = [[tag2idx[t] for t in l] for l in train_labels]
y_train = pad_sequences(maxlen=max_len, sequences=y_train, padding="post", value=tag2idx["O"])
y_train = [to_categorical(i, num_classes=n_tags) for i in y_train]

try:
    with tf.device(device):
        
        # Define the model architecture
        model = Sequential()
        model.add(Embedding(input_dim=n_words, output_dim=10, input_length=max_len))
        model.add(LSTM(units=20, return_sequences=True, recurrent_dropout=0.1))
        model.add(Dropout(0.5))  # Added dropout for regularization
        model.add(Dense(n_tags, activation="softmax"))
        
        # Compile model
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        
        # print model summary
        print(model.summary())
        plot_model(model)

        # Define early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=2)

        # Define model checkpointing
        checkpoint = ModelCheckpoint('model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        
        # Larger batch size
        batch_size = 256  # Adjust based on your GPU memory

        # Train the model
        model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=5, validation_split=0.1, verbose=1, callbacks=[early_stopping, checkpoint])

except Exception as e:
    logging.error("Exception occurred", exc_info=True)
    traceback.print_exc()

#This code sets up a basic LSTM-based model for NER, tokenizes words and labels, pads sequences, defines the model architecture, compiles the model, and finally trains it.

# Prepare the test data
X_test = [[word2idx.get(w, n_words-1) for w in s] for s in test_sentences]
X_test = pad_sequences(maxlen=max_len, sequences=X_test, padding="post", value=n_words-1)

y_test = [[tag2idx.get(t, tag2idx["O"]) for t in l] for l in test_labels]
y_test = pad_sequences(maxlen=max_len, sequences=y_test, padding="post", value=tag2idx["O"])
y_test = [to_categorical(i, num_classes=n_tags) for i in y_test]

# Predict on the test data
y_pred = model.predict(X_test)

# Convert the index to tag
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out

pred_labels = pred2label(y_pred)
test_labels = pred2label(y_test)

# Flatten the lists of labels and predictions
flat_test_labels = [label for sublist in test_labels for label in sublist]
flat_pred_labels = [label for sublist in pred_labels for label in sublist]

# Print the classification report
#print(classification_report(test_labels, pred_labels))

# Calculate the F1 score
f1 = f1_score(flat_test_labels, flat_pred_labels, average='weighted')

print(f'F1 Score: {f1}')

2024-05-14 19:19:10.309173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 19:19:10.309275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 19:19:10.430485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is available!!
          Sentence #           Word Tag
0        Sentence: 1      Thousands   O
1                NaN             of   O
2                NaN  demonstrators   O
3                NaN           have   O
4                NaN        marched   O
...              ...            ...  ..
1048570          NaN           they   O
1048571          NaN      responded   O
1048572          NaN             to   O
1048573          NaN            the   O
1048574          NaN         attack   O

[1048575 rows x 3 columns]
Preprocessing is Completed !!
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 10)            351780    
                                                                 
 lstm (LSTM)                 (None, 50, 20)            2480      
                                                                 
 dropout (Dropout)         

I0000 00:00:1715714447.820048     110 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2252/2252 [==============================] - ETA: 0s - loss: 0.0742 - accuracy: 0.9976
Epoch 1: val_loss improved from inf to 0.00102, saving model to model.keras
2252/2252 [==============================] - 273s 119ms/step - loss: 0.0742 - accuracy: 0.9976 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 2/5
2252/2252 [==============================] - ETA: 0s - loss: 0.0016 - accuracy: 0.9998
Epoch 2: val_loss improved from 0.00102 to 0.00086, saving model to model.keras
2252/2252 [==============================] - 263s 117ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 8.6432e-04 - val_accuracy: 0.9998
Epoch 3/5
2252/2252 [==============================] - ETA: 0s - loss: 0.0010 - accuracy: 0.9998
Epoch 3: val_loss improved from 0.00086 to 0.00080, saving model to model.keras
2252/2252 [==============================] - 267s 119ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 7.9688e-04 - val_accuracy: 0.9998
Epoch 4/5
2252/2252 [==============================] - ETA: 0s -